   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.4 MB 5.9 MB/s eta 0:00:01
   --- ------------------------------------ 0.3/3.4 MB 5.0 MB/s eta 0:00:01
   ------- -------------------------------- 0.7/3.4 MB 4.6 MB/s eta 0:00:01
   ------------ --------------------------- 1.0/3.4 MB 6.0 MB/s eta 0:00:01
   ----------------- ---------------------- 1.5/3.4 MB 6.9 MB/s eta 0:00:01
   --------------------- ------------------ 1.8/3.4 MB 6.7 MB/s eta 0:00:01
   ------------------------- -------------- 2.1/3.4 MB 6.8 MB/s eta 0:00:01
   ------------------------- -------------- 2.2/3.4 MB 6.0 MB/s eta 0:00:01
   ----------------------------- ---------- 2.5/3.4 MB 6.0 MB/s eta 0:00:01
   ---------------------------------- ----- 2.9/3.4 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------  3.4/3.4 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 3.4/3.4 MB 6.3 MB/s eta 0:00:00
   ----------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
def localize_objects(path):
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    objects = client.object_localization(image=image).localized_object_annotations

    print(f"Number of objects found: {len(objects)}")
    for object_ in objects:
        print(f"\n{object_.name} (confidence: {object_.score})")
        print("Normalized bounding polygon vertices: ")
        for vertex in object_.bounding_poly.normalized_vertices:
            print(f" - ({vertex.x}, {vertex.y})")

In [2]:
path = 'C:\\Users\\user\\Desktop\\BigF\\team5\\Detection\\test_crawling_data\\FlamakerConvertibleSectionalSofaCouchModernFabricUShapedLivingRoomFurnitureSetSeatSectionalSleeperSofawithDoubleChaiseMemoryFoamGrey.jpg'
localize_objects(path)

Number of objects found: 4

Couch (confidence: 0.821620523929596)
Normalized bounding polygon vertices: 
 - (0.0546417199075222, 0.3744741678237915)
 - (0.9631007313728333, 0.3744741678237915)
 - (0.9631007313728333, 0.894138753414154)
 - (0.0546417199075222, 0.894138753414154)

Houseplant (confidence: 0.7138032913208008)
Normalized bounding polygon vertices: 
 - (0.004475255962461233, 0.30687347054481506)
 - (0.17322003841400146, 0.30687347054481506)
 - (0.17322003841400146, 0.6499773859977722)
 - (0.004475255962461233, 0.6499773859977722)

Cabinetry (confidence: 0.694159746170044)
Normalized bounding polygon vertices: 
 - (0.4712080955505371, 0.20821700990200043)
 - (0.8000077605247498, 0.20821700990200043)
 - (0.8000077605247498, 0.40176960825920105)
 - (0.4712080955505371, 0.40176960825920105)

Table (confidence: 0.5663102269172668)
Normalized bounding polygon vertices: 
 - (0.6200291514396667, 0.7001201510429382)
 - (0.9538989067077637, 0.7001201510429382)
 - (0.9538989067077637, 